In [1]:
import pandas as pd
import numpy as np
import os
import regex as re
import spacy as sy
import string
from urllib.parse import urlparse
from nltk.tokenize import TweetTokenizer

nlp_en = sy.load('en_core_web_sm')
all_stopwords = nlp_en.Defaults.stop_words

In [2]:
dir_name = '/Users/arjunkhanchandani/Desktop/semeval2023task3bundle-v4/data/en/test-articles-subtask-2'

test_sub2_df = pd.DataFrame()

numbers = list()
headlines = list()
articles = list()

for file in os.walk(dir_name):
    for filename in file[2]:
        
        number = re.findall('[0-9]+', filename)
        numbers.append(number[0])
        x = dir_name + '/' + filename
        # print(number)
        article = ''
        with open(x) as f:
            lines = f.readlines()
            for i in range(len(lines)):
                if i==0:
                    headline = lines[0]
                    headlines.append(headline)
                elif lines[i]=="\n":
                    continue
                else:
                    # print()
                    article = article + '' + lines[i]
        articles.append(article)
            
test_sub2_df['id'] = numbers
test_sub2_df['headlines'] = headlines
test_sub2_df['articles'] = articles

article_df = test_sub2_df.astype({'id': 'int32'})

In [3]:
test_df = test_sub2_df

test_df

,id,headlines,articles
0,3121,The Pope Says 'Humanity Must Repent For Abusin...,We humans “must repent and modify our lifestyl...
1,3135,Russian allies are unknowingly working against...,The Secretary of State for Defence described P...
2,3134,"I know this week has been disruptive, says LIZ...",The United Kingdom is the greatest country on ...
3,3120,Vladimir Putin facing Ukraine 'humiliation' as...,"VLADIMIR PUTIN is facing ""humiliation"" in Ukra..."
4,3136,What Are the Odds Putin is Bluffing About Usin...,Eurointelligence founder Wolfgang Münchau has ...
5,3122,Florida Surgeon General Issues Warning For mRN...,"Florida Surgeon General Joseph A. Ladapo, MD, ..."
6,3123,Biden's Secret Promise To OPEC Backfires: Shel...,"In early September, United States Secretary of..."
7,3137,Central Banks Add Gold for Fifth Straight Month\n,Central banks globally added to their net gold...
8,3133,US denies 'preposterous' claim they 'sabotaged...,It comes as Donald Trump suggested that 'World...
9,3127,"Did Uncle Sam, a.k.a. Wile E. Coyote, Blow Up ...",After “parties not-so-unknown” bombed the Nord...


In [4]:
def preprocessing(x, y, df):
    
    pos_tags_final_text = list()
    er_final_text = list()
    preprocessed_text = list()

    for x in df.loc[:,y]:

        tokenizer = TweetTokenizer()
        #tokenizing
        doc = tokenizer.tokenize(x)
        
        # removing links
        tokens = [token for token in doc if not urlparse(token).scheme]
        x = ' '.join(tokens)
        doc = nlp_en(x)
        
        # removing punctuation and white space
        tokens = [token.orth_ for token in doc if not token.is_punct | token.is_space]    
        x = ' '.join(tokens)
        
        # lower case
        x = x.lower()
        doc = nlp_en(x)

        # lemmatization
        tokens = [word.lemma_ for word in doc]   
        x = ' '.join(tokens)
        doc = nlp_en(x)  
        
        # removing punctuation and white space
        tokens = [token.orth_ for token in doc if not token.is_punct | token.is_space]    
        x = ' '.join(tokens)
        doc = nlp_en(x)
        
        # removing individual letters
        tokens = [word.text for word in doc if len(word)>=2]
        x = ' '.join(tokens)  
        # print(x)
        doc = nlp_en(x)
        
        # removing stop words
        tokens = [word for word in doc if not word in all_stopwords]
        list_of_strings  = [i.text for i in tokens]
        x = ' '.join(list_of_strings)
        doc = nlp_en(x)
        
        # Part of speech tagging
        pos_tags = [(i, i.tag_) for i in doc]
        pos_tags_final_text.append(pos_tags)
        
        # entity recognition tagging
        er =  [(i, i.label_, i.label) for i in doc.ents] 
        er_final_text.append(er)
        
        preprocessed_text.append(x)
        
    return pos_tags_final_text, er_final_text, preprocessed_text

In [6]:
pos_tags_final_headlines, er_final_headlines, preprocessed_headlines = preprocessing(headline, 'headlines', test_df)

test_df['preprocessed_headlines'] = preprocessed_headlines
test_df['pos_tags_headlines'] = pos_tags_final_headlines
test_df['er_tags_headlines'] = er_final_headlines

In [7]:
pos_tags_final_articles, er_final_articles, preprocessed_articles = preprocessing(article, 'articles', test_df)

test_df['preprocessed_articles'] = preprocessed_articles
test_df['pos_tags_articles'] = pos_tags_final_articles
test_df['er_tags_articles'] = er_final_articles

In [8]:
test_df.head()

,id,headlines,articles,preprocessed_headlines,pos_tags_headlines,er_tags_headlines,preprocessed_articles,pos_tags_articles,er_tags_articles
0,3121,The Pope Says 'Humanity Must Repent For Abusin...,We humans “must repent and modify our lifestyl...,the pope say humanity must repent for abuse mo...,"[(the, DT), (pope, NN), (say, VBP), (humanity,...",[],we human must repent and modify our lifestyle ...,"[(we, PRP), (human, NN), (must, MD), (repent, ...","[((thursday), DATE, 391), ((the, world, day), ..."
1,3135,Russian allies are unknowingly working against...,The Secretary of State for Defence described P...,russian ally be unknowingly work against they ...,"[(russian, JJ), (ally, NN), (be, VB), (unknowi...","[((russian), NORP, 381)]",the secretary of state for defence describe pu...,"[(the, DT), (secretary, NNP), (of, IN), (state...","[((putin), PERSON, 380), ((russian), NORP, 381..."
2,3134,"I know this week has been disruptive, says LIZ...",The United Kingdom is the greatest country on ...,know this week have be disruptive say liz truss,"[(know, VB), (this, DT), (week, NN), (have, VB...","[((this, week), DATE, 391)]",the united kingdom be the great country on ear...,"[(the, DT), (united, NNP), (kingdom, NNP), (be...","[((the, united, kingdom), GPE, 384), ((70, yea..."
3,3120,Vladimir Putin facing Ukraine 'humiliation' as...,"VLADIMIR PUTIN is facing ""humiliation"" in Ukra...",vladimir putin face ukraine humiliation as rus...,"[(vladimir, NNP), (putin, NNP), (face, NNP), (...","[((vladimir, putin), PERSON, 380), ((ukraine),...",vladimir putin be face humiliation in ukraine ...,"[(vladimir, NNP), (putin, NNP), (be, VB), (fac...","[((vladimir, putin), PERSON, 380), ((ukraine),..."
4,3136,What Are the Odds Putin is Bluffing About Usin...,Eurointelligence founder Wolfgang Münchau has ...,what be the odd putin be bluff about use nucle...,"[(what, WP), (be, VB), (the, DT), (odd, JJ), (...",[],eurointelligence founder wolfgang münchau have...,"[(eurointelligence, NN), (founder, NN), (wolfg...","[((wolfgang, münchau), PERSON, 380), ((putin),..."


In [9]:
# converting tweets_df into a csv file
filename = '/Users/arjunkhanchandani/Desktop/semeval2023task3bundle-v4/test_subtask_2.csv'
test_df.to_csv(filename, index=True)